In [1]:
!conda info -e

# conda environments:
#
base                     /Users/FUZZY/opt/anaconda3
tensor                *  /Users/FUZZY/opt/anaconda3/envs/tensor



In [2]:
# !pip list -o

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.datasets import mnist
import scipy.stats

# コードリーディング

TensorFLowによって2値分類を行うサンプルコードを載せました。今回はこれをベースにして進めます。


tf.kerasやtf.estimatorなどの高レベルAPIは使用していません。低レベルなところから見ていくことにします。

## 【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。


（例）


* 重みを初期化する必要があった
* エポックのループが必要だった

それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。

データセットの用意
以前から使用しているIrisデータセットを使用します。以下のサンプルコードではIris.csvが同じ階層にある想定です。


[Iris Species](https://www.kaggle.com/uciml/iris/data)


目的変数は`Species`ですが、3種類ある中から以下の2種類のみを取り出して使用します。

Iris-versicolor
Iris-virginica

## 【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。


それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。


《サンプルコード》


＊バージョン1.5から1.14の間で動作を確認済みです。

In [12]:
# # データセットの読み込み
# from sklearn.datasets import load_iris
# iris_data = load_iris()
# X = iris_data.data[50:]
# y = iris_data.target[50:]
# print(y.shape)
# y = y.astype(np.int)[:, np.newaxis]
# print(y.shape)

In [13]:
# """
# TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
# """
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# import tensorflow as tf



# # y = y.astype(np.int)[:, np.newaxis]
# # trainとtestに分割
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# # さらにtrainとvalに分割
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# # ハイパーパラメータの設定
# learning_rate = 0.01
# batch_size = 10
# num_epochs = 10
# n_hidden1 = 50
# n_hidden2 = 100
# n_input = X_train.shape[1]
# n_samples = X_train.shape[0]
# n_classes = 1
# # 計算グラフに渡す引数の形を決める
# X = tf.placeholder("float", [None, n_input])
# Y = tf.placeholder("float", [None, n_classes])
# # trainのミニバッチイテレータ
# get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
# def example_net(x):
#     """
#     単純な3層ニューラルネットワーク
#     """
#     # 重みとバイアスの宣言
#     weights = {
#         'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
#         'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
#         'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
#     }
#     biases = {
#         'b1': tf.Variable(tf.random_normal([n_hidden1])),
#         'b2': tf.Variable(tf.random_normal([n_hidden2])),
#         'b3': tf.Variable(tf.random_normal([n_classes]))
#     }
#     layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
#     layer_1 = tf.nn.relu(layer_1)
#     layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
#     layer_2 = tf.nn.relu(layer_2)
#     layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
#     return layer_output
# # ネットワーク構造の読み込み                               
# logits = example_net(X)
# # 目的関数
# loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# # 最適化手法
# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
# train_op = optimizer.minimize(loss_op)
# # 推定結果
# correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# # 指標値計算
# accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# # variableの初期化
# init = tf.global_variables_initializer()

# # 計算グラフの実行
# with tf.Session() as sess:
#     sess.run(init)
#     for epoch in range(num_epochs):
#         # エポックごとにループ
#         total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int) # np.ceil:小数点切り上げ
#         total_loss = 0
#         total_acc = 0
#         for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
#             # ミニバッチごとにループ
#             sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
#             loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
#             total_loss += loss
#             total_acc += acc
#         total_loss /= n_samples
#         total_acc /= n_samples
#         val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
#         print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
#     test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
#     print("test_acc : {:.3f}".format(test_acc))

# 他のデータセットへの適用

これまで扱ってきた小さなデータセットがいくつかあります。上記サンプルコードを書き換え、これらに対して学習・推定を行うニューラルネットワークを作成してください。


* Iris（3種類全ての目的変数を使用）
* House Prices

どのデータセットもtrain, val, testの3種類に分けて使用してください。

## 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。


Iris Species


2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。


《ヒント》


以下の2箇所は2クラス分類特有の処理です。

In [14]:
# loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))

In [15]:
# correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))

メソッドは以下のように公式ドキュメントを確認してください。


[tf.nn.sigmoid_cross_entropy_with_logits  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/nn/sigmoid_cross_entropy_with_logits)


[tf.math.sign  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/math/sign)


＊tf.signとtf.math.signは同じ働きをします。

### 【問題3】解答

In [16]:
# データセットの読み込み
from sklearn.datasets import load_iris
iris_data = load_iris()
X = iris_data.data
y = iris_data.target

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y = enc.fit_transform(y[:, np.newaxis])
y.shape

(150, 3)

In [17]:
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [18]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf


# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = y_train.shape[1]
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int) # np.ceil:小数点切り上げ
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

AttributeError: module 'tensorflow' has no attribute 'placeholder'

### 【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。


[House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)


この中の`train.csv`をダウンロードし、目的変数として`SalePrice`、説明変数として、`GrLivArea`と`YearBuilt`を使ってください。説明変数はさらに増やしても構いません。


分類問題と回帰問題の違いを考慮してください。

In [11]:
import pandas as pd

df= pd.read_csv('../house-prices-advanced-regression-techniques/train.csv')
X = df[['GrLivArea', 'YearBuilt']]
y = df.SalePrice[:, np.newaxis]
y.shape

(1460, 1)

In [12]:
X = X.values
# 正規化
X = scipy.stats.zscore(X)
y = scipy.stats.zscore(y)
# # trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [13]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf


# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.square(Y - logits)) # MSE
# loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = logits
# 指標値計算
accuracy = tf.reduce_mean(tf.square(Y - logits))
# accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int) # np.ceil:小数点切り上げ
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, MSE : {:.3f}, val_MSE : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_MSE : {:.3f}".format(test_acc))

Epoch 0, loss : 5.3977, val_loss : 7.5562, MSE : 5.398, val_MSE : 7.556
Epoch 1, loss : 0.9188, val_loss : 3.6849, MSE : 0.919, val_MSE : 3.685
Epoch 2, loss : 2.8245, val_loss : 2.7752, MSE : 2.825, val_MSE : 2.775
Epoch 3, loss : 1.3559, val_loss : 1.3771, MSE : 1.356, val_MSE : 1.377
Epoch 4, loss : 0.6327, val_loss : 1.0268, MSE : 0.633, val_MSE : 1.027
Epoch 5, loss : 0.2613, val_loss : 1.2510, MSE : 0.261, val_MSE : 1.251
Epoch 6, loss : 0.3260, val_loss : 1.8946, MSE : 0.326, val_MSE : 1.895
Epoch 7, loss : 0.3362, val_loss : 2.1175, MSE : 0.336, val_MSE : 2.118
Epoch 8, loss : 0.9631, val_loss : 4.2581, MSE : 0.963, val_MSE : 4.258
Epoch 9, loss : 0.6165, val_loss : 2.4943, MSE : 0.616, val_MSE : 2.494
test_MSE : 3.610


### 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。


3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。


スクラッチで実装したモデルの再現を目指してください。

In [22]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train[:,:,:,np.newaxis]
X_test = X_test[:,:,:,np.newaxis]
# # NCHW
# X_train = X_train[:,:,:,np.newaxis]
# X_test = X_test[:,:,:,np.newaxis]

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = enc.fit_transform(y_train[:, np.newaxis])
y_test = enc.transform(y_test[:, np.newaxis])

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [23]:
X_train.shape, y_train.shape

((48000, 28, 28, 1), (48000, 10))

In [40]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf


# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100

# n_input = X_train.shape[1]
n_samples = X_train.shape[0]
ch = X_train.shape[3]
height = X_train.shape[1]
width = X_train.shape[2]

# strides = 1
# padding = 0


n_classes = y_train.shape[1]

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, height, width, ch])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([3, 3, ch, 32])),
#         'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([6272, 10]))
               }
    biases = {
        'b1': tf.Variable(tf.random_normal([ch])),
#         'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([10]))
              }


    layer_1 = tf.nn.conv2d(input=x, filter=weights['w1'], padding="SAME", strides=[1, 1], data_format='NHWC')
    print(layer_1)
    layer_1_1 = tf.nn.relu(layer_1)
    layer_2 = tf.nn.max_pool(layer_1_1, ksize=[2, 2], strides=[2, 2], padding="SAME")
    print(layer_2.shape)
    layer_3 = tf.reshape(layer_2, [-1, layer_2.shape[1]*layer_2.shape[2]*layer_2.shape[3]]) # Flatten
    print(layer_3.shape)
    layer_output = tf.matmul(layer_3, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int) # np.ceil:小数点切り上げ
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Tensor("Conv2D_20:0", shape=(?, 28, 28, 32), dtype=float32)
(?, 14, 14, 32)
(?, 6272)
Epoch 0, loss : 0.0401, val_loss : 0.2430, acc : 1.000, val_acc : 0.940
Epoch 1, loss : 0.0306, val_loss : 0.1293, acc : 1.000, val_acc : 0.960
Epoch 2, loss : 0.0006, val_loss : 0.1122, acc : 1.000, val_acc : 0.969
Epoch 3, loss : 0.0107, val_loss : 0.1375, acc : 1.000, val_acc : 0.966
Epoch 4, loss : 0.0024, val_loss : 0.1672, acc : 1.000, val_acc : 0.965
Epoch 5, loss : 0.0003, val_loss : 0.2038, acc : 1.000, val_acc : 0.965
Epoch 6, loss : 0.0001, val_loss : 0.2041, acc : 1.000, val_acc : 0.969
Epoch 7, loss : 0.0259, val_loss : 0.2464, acc : 1.000, val_acc : 0.969
Epoch 8, loss : 0.0002, val_loss : 0.2908, acc : 1.000, val_acc : 0.966
Epoch 9, loss : 0.0123, val_loss : 0.2825, acc : 1.000, val_acc : 0.968
test_acc : 0.968
